# Cvičení 3

Tématem tohoto cvičení jsou přímé metody řešení soustav lineárních rovnic. Budeme se zabývat dopřednou a zpětnou substitucí a LU rozkladem.

## Dopředná a zpětná substituce

Základní princip přímých metod řešení soustav lineárních rovnic je převod matice soustavy na součin trojúhelníkových matic, např. 
$$\mathsf{A} = \mathsf{L}\mathsf{U}.$$
Soustavu $\mathsf{A}\mathbf{x} = \mathbf{b}$ tak převedeme na tvar
$$\mathsf{L}\mathsf{U}\mathbf{x} = \mathbf{b}$$
a vyřešíme ji ve dvou krocích 
1) $\mathsf{L}\mathbf{y}=\mathbf{b}$
2) $\mathsf{U}\mathbf{x}=\mathbf{y}$
pomocí dopředné a zpětné substituce.

V této části cvičení tedy nejdříve implementujeme funkce pro dopřednou a zpětnou substituci, které později využijeme k řešení soustav.

### Testovací matice

Nejprve vygenerujte testovací matice, pomocí kterých později zjistíte, zda vaše funkce pracují správně. Budete k tomu potřebovat
- funkci z knihovny NumPy, pomocí které vygenerujete náhodnou matici o daném rozměru (znáte z prvního cvičení),
- funkce `triu`, `tril` z knihovny NumPy (zjistěte v dokumentaci nebo pomocí Googlu, k čemu slouží a jak se používají)

In [1]:
# ÚKOL: importujte potřebnou knihovnu NumPy pod zkratkou np
import numpy as np

In [2]:
# ÚKOL: vygenerujte matici A náhodných čísel o rozměrech 4x4
A = np.random.rand(4,4)

np.set_printoptions(precision=3) # pro lepší přehlednost
print(A)

[[0.759 0.65  0.891 0.847]
 [0.945 0.826 0.644 0.025]
 [0.644 0.292 0.218 0.242]
 [0.229 0.802 0.091 0.011]]


In [3]:
# ÚKOL: pomocí vhodných funkcí z knihovny NumPy extrahujte z matice A její horní a dolní trojúhelníkovou část.
U = np.triu(A,0)
print(U)
print("---------------------")

L = np.tril(A,0)
print(L)

# Ověřte funkčnost porovnáním s příslušnou částí matice A

[[0.759 0.65  0.891 0.847]
 [0.    0.826 0.644 0.025]
 [0.    0.    0.218 0.242]
 [0.    0.    0.    0.011]]
---------------------
[[0.759 0.    0.    0.   ]
 [0.945 0.826 0.    0.   ]
 [0.644 0.292 0.218 0.   ]
 [0.229 0.802 0.091 0.011]]


In [4]:
# ÚKOL: Vytvořte náhodný vektor b délky 4
b = np.random.rand(4,1)

print(b)

[[0.122]
 [0.346]
 [0.572]
 [0.042]]


### Dopředná substituce

Soustavu s dolní trojúhelníkovou maticí $\mathsf{L}\mathbf{x}=\mathbf{b}$ můžeme efektivně řešit pomocí algoritmu dopředné substituce. 

1) Určíme první prvek vektoru řešení
$$x_1 = b_1 / L_{1, 1}$$

2) Zbývající prvky určíme jako 
$$\forall i \in \{2, 3, \ldots, m\}: x_i = (b_i - \sum_{j=1}^{i-1}L_{i, j} x_j) / L_{i, i}$$

Na základě těchto předpisů doplňte chybějící části následujícího kódu. Nezapomeňte ale, že v Pythonu indexujeme od nuly.

In [5]:
# ÚKOL: Doplňte následující kód

def fsubst(L, b):
    m, n = np.shape(L)             # pomocí vhodné funkce numpy uložte do proměnných m, n počet řádků a sloupců L
    x = np.zeros((n,1))                # vytvořte vektor samých nul délky n
    
    x[0][0] = b[0][0] / L[0, 0]   # první prvek řešení

    for i in range(1, m):
        # nejdříve pomocí vnořeného for cyklu určíme hodnotu sumy z kroku 2:
        suma = 0
        for j in range(0,i):                       # doplňte vhodně meze smyčky
            suma += L[i][j]*x[j][0]                        # doplňte
        
        # odečteme sumu od prvku pravé strany a vydělíme diagonálním prvkem
        x[i][0] = (b[i][0] - suma) / L[i][i]                            # doplňte

    return x


In [6]:
# Otestujte pomocí L, b
x = fsubst(L, b)
print(x)

[[  0.161]
 [  0.234]
 [  1.836]
 [-32.153]]


In [7]:
# ÚKOL: určete řešení pomocí funkce solve z numpy a porovnejte s vašim řešením
x_numpy = np.linalg.solve(L,b)
print(x_numpy)

[[  0.161]
 [  0.234]
 [  1.836]
 [-32.153]]


### Zpětná substituce

K vyřešení soustavy s horní trojúhelníkovou maticí $\mathsf{U}\mathbf{x} = \mathbf{b}$ potřebujeme implementovat také algoritmus zpětné substituce. Ten je podobný dopředné substituci, postupuje však od posledního řádku:

1) Určíme poslední prvek vektoru řešení 
$$x_m = b_m / U_{m,m}$$
2) Určíme zbývající prvky
$$\forall i \in \{m-1, m-2, \ldots, 1\}: x_i = (b_i - \sum_{j=i+1}^{m}U_{i, j}x_j) / U_{i, i}$$

Implementujte v následující buňce vaši verzi zpětné substituce.

**Poznámka** Při implementaci se vám bude hodit vědět, že funkce range umožňuje iterovat i "pozpátku". Vyzkoušejte si v následující buňce vytisknout přes jaká *i* iterují tyto `for` cykly:
- `for i in range(6, 0, -1)`
- `for i in range(6, -1, -1)`
- `for i in range(6, 0, -2)`
- `for i in range(0, -6, -1)`

In [8]:
# ÚKOL: Vytiskněte proměnnou i z příkladů v předchozí buňce.
for i in range(0,-6,-1):
    print(i)

0
-1
-2
-3
-4
-5


In [9]:
# ÚKOL: Implementujte algoritmus zpětné substituce.

def b_subst(U, b):
    m, n = np.shape(U)
    x = np.zeros(m)

    x[m-1] = b[m-1][0] / U[m-1][n-1]

    for i in range(m-2, -1, -1):
        suma = 0
        for j in range(i, m):
            suma += U[i][j]*x[j]
    
        x[i] = (b[i][0] - suma) / U[i][i]

    return x
    
    


In [10]:
# ÚKOL: ověřte pomocí numpy, že vaše funkce vrací správný výsledek

x2 = b_subst(U, b)
x2_numpy = np.linalg.solve(U,b) # doplňte
print(x2)
print(x2_numpy)

[-3.585  1.627 -1.702  3.894]
[[-3.585]
 [ 1.627]
 [-1.702]
 [ 3.894]]


# Domácí úkol č. 1

Algoritmus LU rozkladu lze popsat např. následujícím pseudokódem.

```
U = A, L = I
for k = 1 to m-1 do
    for j = k+1 to m do
        L(j, k) = U(j, k)/U(k, k)
        U(j, k:m) = U(j, k:m) - L(j, k)U(k, k:m)
    end for
end for
```

**Pozor**: značení `k:m` zde znamená sloupce od `k` po `m` *včetně*.

Vašim úkolem je:
1) Na základě uvedeného pseudokódu implementovat LU rozklad bez pivotizace do metody `L, U = my_lu(A)`. Metoda tedy bude mít na vstupu čtvercovou matici A a na výstupu dolní trojúhelníkovou matici L a horní trojúhelníkovou matici U takové, že `A = LU`.
2) Sestavit náhodnou matici A o rozměrech 5x5 a pomocí ní ověřit, že vaše metoda funguje správně (`A - L @ U` by mělo vrátit matici s nulovými či velmi malými prvky).
3) Sestavit náhodný vektor pravé strany `b` o délce 5. 
4) Pomocí vaší metody `my_lu` a na cvičení implementovaných metod `f_subst` a `b_subst` vyřešit soustavu `Ax = b`.
5) Ověřit pomocí metody `np.linalg.solve`, že jste nalezli správné řešení. 

**Odevzdání**: Celý tento notebook s řešením zašlete na e-mail vyučujícího do 23. 3. 16:00. Jako předmět e-mailu uveďte "NLA1 DU1".

In [11]:
# Doplňte podle bodu 1)

def my_lu(A):
    m, n = np.shape(A)
    U = A.copy()
    ones = np.ones(m)
    L = np.diag(ones, 0)

    for k in range(0, m-1):
        for j in range(k+1, m):
            L[j][k] = U[j][k] / U[k][k]
            U[j][k:m+1] = U[j][k:m+1] - L[j][k] * U[k][k:m+1]
    return L, U
            
        


In [12]:
# Doplňte podle bodu 2)
A = np.random.rand(5,5)
L, U = my_lu(A)
print(A)
print(A-L@U)


[[0.04  0.786 0.045 0.2   0.744]
 [0.516 0.035 0.187 0.181 0.11 ]
 [0.985 0.982 0.237 0.412 0.32 ]
 [0.5   0.276 0.976 0.203 0.062]
 [0.549 0.763 0.831 0.935 0.652]]
[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -2.220e-16]
 [ 0.000e+00 -1.110e-15  0.000e+00  1.110e-16  1.110e-16]
 [ 5.551e-16 -2.776e-16  1.110e-16  1.110e-16 -7.772e-16]
 [ 4.441e-16  2.220e-16 -1.110e-16 -2.220e-16 -1.665e-15]]


In [13]:
# Doplňte podle bodu 3)
b = np.random.rand(5,1)
print(b)

[[0.913]
 [0.052]
 [0.15 ]
 [0.267]
 [0.454]]


In [14]:
# Doplňte podle bodu 4)
y = fsubst(L,b)
y = y.reshape(5,1)
x = b_subst(U,y)
print(x)

[-0.057 -0.008  0.383 -0.814  1.433]


In [15]:
# Doplňte podle bodu 5)
print(np.linalg.solve(A,b))

[[-0.057]
 [-0.008]
 [ 0.383]
 [-0.814]
 [ 1.433]]
